In [1]:
import numpy as np
from py3gpp import *

In [2]:
A = 10000           # Transport block length, positive integer
rate = 449/1024     # Target code rate, 0<R<1
rv = 0              # Redundancy version, 0-3
modulation = 'QPSK'
nlayers = 1

cbs_info = nrDLSCHInfo(A, rate)
print(cbs_info)

{'CRC': '24A', 'L': 24, 'BGN': 1, 'C': 2, 'Lcb': 24, 'F': 244.0, 'Zc': 240, 'K': 5280, 'N': 15840}


In [3]:
in_ = np.random.randint(0, 2, (A, 1))

tb_in = nrCRCEncode(in_, cbs_info['CRC'])

cbs_in = nrCodeBlockSegmentLDPC(tb_in, cbs_info['BGN'])

enc = nrLDPCEncode(cbs_in, cbs_info['BGN'])

outlen = np.ceil(A/rate)
ch_in = nrRateMatchLDPC(enc, outlen, rv, modulation, nlayers)

In [14]:
ch_out = double(1-2*(ch_in))
raterec = nrRateRecoverLDPC(ch_out, A, rate, rv, modulation, nlayers)

dec_bits, iters = nrLDPCDecode(raterec, cbs_info['BGN'], 25)
print(f'decoding finished after {iters} iterations')

blk, blk_err = nrCodeBlockDesegmentLDPC(dec_bits, cbs_info['BGN'], A + cbs_info['L'])
print(f'code block CRC: {int(blk_err)}')

out, tb_err = nrCRCDecode(blk, cbs_info['CRC'])
print(f'transport block CRC: {tb_err}')

np.array_equal(in_, out)

block length B = 16320, number of information bits K = 5280, code rate R = 0.3333333333333333, i_ls = 7
decoding segment 0 ...


decoding segment 1 ...
decoding finished after 25 iterations
code block CRC: 0
transport block CRC: [0]


Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 1134, in _pydevd_bundle.pydevd_cython.PyDBFrame.trace_dispatch
  File "_pydevd_bundle/pydevd_cython.pyx", line 311, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/home/benja/.local/lib/python3.10/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2062, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/home/benja/.local/lib/python3.10/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2098, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


KeyboardInterrupt: 